# FLOWCELL | EXPERIMENT

[YYYY-mm-dd] CopyComplete.txt 'mmm dd hh:mm'

```bash
screen -S sign

export FC=""
export PI="Sinnett_DS"
export BASEDIR="/mnt/vs_nas_chusj/CQGC_PROD/sequenceurs/${a[1]}"
export WORKDIR="/mnt/vs_nas_chusj/CQGC_PROD/fastqs"
export LOGIN="hien"

conda activate CQGC-bcl_convert
python /staging2/soft/CQGC-utils/Helpers/get_nanuq_files.py

ls -l ${BASEDIR}/${FC}/CopyComplete.txt 
# => file size is zero and NO "File not found" error message
``````

In [ ]:
# 6.3.1 If only one index size is present in the SampleSheet. Otherwise, split the SampleSheet

cat ${WORKDIR}/${FC}/SampleSheet.csv
qsub -V /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/dragen_bcl-convert_launcher.sh ${FC}

qstat -u ${USER}
tail -f ${WORKDIR}/${FC}/${FC}.bcl-convert.log


In [ ]:
# Split SampleSheet by index and note index size for each lane 
# _N.B._ Run logs for both index written to the same file 
python /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/split_samples_by_index_size.py ${WORKDIR}/${FC}/SampleSheet.csv 

i=8 # or i=10
mv ${WORKDIR}/${FC}/SampleSheet-i${i}.csv ${WORKDIR}/${FC}/SampleSheet.csv
qsub -V /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/dragen_bcl_convert.sh
# Wait for job to finish, then do some post-processing
# before moving to the next size of index
mv ${WORKDIR}/${FC}/1.fastq/Logs ${WORKDIR}/${FC}/1.fastq/Logs-i${i}
mv ${WORKDIR}/${FC}/1.fastq/Reports ${WORKDIR}/${FC}/1.fastq/Reports-i${i}
python /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/assemble_report.py --pools ${WORKDIR}/${FC}/SamplePools.csv ${WORKDIR}/${FC}/1.fastq/Reports-i${i} 

i=10
mv ${WORKDIR}/${FC}/SampleSheet-i${i}.csv ${WORKDIR}/${FC}/SampleSheet.csv
qsub -V /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/dragen_bcl_convert.sh
# Wait for job to finish, then do some post-processing
mv ${WORKDIR}/${FC}/1.fastq/Logs ${WORKDIR}/${FC}/1.fastq/Logs-i${i}
mv ${WORKDIR}/${FC}/1.fastq/Reports ${WORKDIR}/${FC}/1.fastq/Reports-i${i}
python /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/assemble_report.py --pools ${WORKDIR}/${FC}/SamplePools.csv ${WORKDIR}/${FC}/1.fastq/Reports-i${i} 


In [ ]:
python /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/assemble_report.py --pools ${WORKDIR}/${FC}/SamplePools.csv ${WORKDIR}/${FC}/1.fastq/Reports
bash /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/cp_RunInfo_Stats.sh
rsync -uva --progress ${WORKDIR}/${FC} ${LOGIN}@beluga.calculquebec.ca:/scratch/${LOGIN}/_fastq/${PI}/

In [ ]:
export FC="241004_A00516_0603_AHJKJWDRX5"
a=($(echo ${FC} | tr '_' '\n'))
export PI="Sinnett_DS"
export LOGIN="hien"

cd ${SCRATCH}/_fastq/${PI}/${FC}/1.fastq/
mv Sinnett_SIGNATURE/*.fastq.gz .
rmdir Sinnett_SIGNATURE
python ${HOME}/projects/def-rallard/COMMUN/scripts/rename_samples.py ${SCRATCH}/_fastq/${PI}/${FC}

rm -R ${SCRATCH}/_fastq/${PI}/${FC}/1.fastq/Logs*
rm ${SCRATCH}/_fastq/${PI}/${FC}/Sample*
mv ${SCRATCH}/_fastq/${PI}/${FC} ${SCRATCH}/Globus/${PI}


### Livraison

[2024-06-10] Attention: Ne pas suivre la procédure pour la livraison car les échantillons se retrouvent sur 2 runs et certains échantillons vont au projet PROFYLE et non pas au projet SIGNATURE.

    From: Sylvie Langlois (HSJ) <sylvie.langlois.hsj@ssss.gouv.qc.ca>
    Sent: June 10, 2024 15:07
    To: Quang-Hien Le (HSJ) <quang-hien.le.hsj@ssss.gouv.qc.ca>
    Cc: Charles Prive (HSJ) <charles.prive.hsj@ssss.gouv.qc.ca>
    Subject: Re: PROFYLE ou SIGNATURE
    
    Bonjour,
    Seul ceux-ci vont au groupe PROFYLE:
    186617_PRO-00929_N_WGS_blood_45454
    186617_PRO-00929_T2_WGS_carcinoma_45448
    170150_PRO-00733_N_WGS_blood_45473
    170150_PRO-00733_R3_WGS_carcinomatosis_45426

    Les autres vont sur Signature.

In [ ]:
# ssh beluga

FC="20240603_LH00336_0056_B223CLHLT1"
rsync -uva --progress --exclude *.fastq.gz ${SCRATCH}/Globus/Sinnett_DS/${FC} ${SCRATCH}/Globus/PROFYLE/
mv ${SCRATCH}/Globus/Sinnett_DS/${FC}/1.fastq/*PRO*.fastq.gz ${SCRATCH}/Globus/PROFYLE/${FC}/1.fastq/
rm -R ${SCRATCH}/Globus/PROFYLE/${FC}/1.fastq/Reports-i8

FC="20240607_LH00336_0060_B223CFNLT1"
rsync -uva --progress --exclude *.fastq.gz ${SCRATCH}/Globus/Sinnett_DS/${FC} ${SCRATCH}/Globus/PROFYLE/
mv ${SCRATCH}/Globus/Sinnett_DS/${FC}/1.fastq/*PRO*.fastq.gz ${SCRATCH}/Globus/PROFYLE/${FC}/1.fastq/


In [ ]:
# Edit the delivery message so that it is adapted for SIGNATURE or PROFYLE recipients

echo "Objet:[Dr ${PI}] ${FC} 
--------- 
Les fichiers .FASTQ de votre projet ${FC} sont maintenant accessibles depuis ce lien Globus : 

​https://app.globus.org/file-manager?origin_id=7574e08a-ff32-11ea-81a3-0e2f230cc907 

​DRAGEN Version 4.2.4 a été utilisé pour ce procédé de déconvolution et conversion. Sous le répertoire ${FC} se trouvent: 

- Les rapports statistiques de déconvolution des reads ./1.fastq/Reports/Demultiplex_Stats.csv 
- La feuille des échantillons ./1.fastq/Reports/SampleSheet.csv
- Les paramètres utilisés lors du procédé de conversion et de déconvolution des reads ./${FC}.bcl-convert.log 
- Une liste des fichiers de séquences compressées (*.fastq.gz) ./1.fastq/Reports/fastq_list.csv 

Aucun filtre n'a été appliqué sur les séquences: les séquences des adaptateurs sont encore présentes l'outil Trimmomatic http://www.usadellab.org/cms/?page=trimmomatic peut être utilisé pour appliquer les filtres de trimming des reads selon les paramètres de votre choix. 

​Les fichiers et le rapport statistique ont été renommés à votre demande.
 
​Cordialement,
​" | tee ${SCRATCH}/Globus/${PI}/${FC}/delivery_msg.txt



In [ ]:
# 6.5.1 Archivage de FASTQ
# ssh spxp-app02 && setup env 

cd ${WORKDIR}
rm ${WORKDIR}/${FC}/1.fastq/Undetermined_*
find ${WORKDIR}/${FC}/ -type f -regex ".*/*fastq.gz" -exec sha1sum -b {} \; >> ${WORKDIR}/${FC}.sha1
tar -cvzf ${FC}.tar.gz ${FC} > ${FC}.tar.gz.log
sha1sum ${FC}.tar.gz > ${FC}.tar.gz.sha1
rsync -uva --progress ${FC}.* ${LOGIN}@beluga.calculquebec.ca:/nearline/rpp-montpea/CIGCP_COMMUN/_reads_GQSJ/IlluminaNovaSeq6000_fastq/${PI}/ && touch me


In [ ]:
# 6.5.2 ​Vérifier l'intégrité des transferts 
# ssh spxp-app02 && setup env 

export FC="240816_A00516_0589_BHVGF5DMXY"
a=($(echo ${FC} | tr '_' '\n'))
export PI="Sinnett_DS"
export LOGIN="hien"

cd ${SCRATCH}
sbatch ~/bin/CQGC-utils/Analysis.dragen_bcl-convert/scripts/checksum.sh 


In [ ]:
Set-Location 'D:\HSJ\Projects\Process Seq Runs\Runs\240816_A00516_0589_BHVGF5DMXY'
jupyter nbconvert --to html --template lab .\README-A00516_0589.ipynb
